In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import requests
import json
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain import PromptTemplate
from groq import Groq
import PyPDF2
from openai import AzureOpenAI
import openai
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

load_dotenv()

True

In [66]:
# # Define the LangChain prompt template with placeholders
# template = """
# By using the context below, please explain the additional market condition and sentiment that would affect this company specifically to this segment of the busisness.

# Context:
# {context}

# Please generate the response.
# """

# # Create the PromptTemplate
# prompt_template = PromptTemplate(
#     input_variables=["context"],
#     template=template,
# )

# # Example old and new code snippets
# # context = "A negative industrial outlook is support by reason of the demand of glove increase during the pandemic. As more glove is needed during the pandemic which cause the price and earning of Top Glove to Increase. As evidence, by company manager Lee Kim Meow stated that the goods are all sold out to their distributors and customers even before the supplies have arrived at Top-Glove warehouse. As the vaccination plan all over the world is being ramp up and being administer to more and more people which mean that the Covid-19 pandemic is being more and more in control. This would lead to decrease in the earning for Top-glove back to its average earnings before the pandemic. According to Top-Glove presentation in 2018 (refer to chart 1), Top Glove key market include Europe, North America and Asia make up Top Glove sales by 31%, 25% and 23% respectively. As USA have a sharp decrease in Covid-19 cases as of today using a 7 day average USA have around 12k of new Covid-19 cases while they are administering the vaccine at the doses per day of around 1 million. This means that the vaccine is being administer at a rate that the virus is infecting people slower than the vaccine is being administer.Furthermore in Europe, Germany is also show that the Covid-19 cases have been similar to the flatten peak from the highest at 15399 new cases at 25 of April 2021 to 903 at 8 of July 2021which means they overcome their second wave with the vaccine being administer in Germany at a rate about 600k to 800k per day . Meanwhile in China the curve have always been flatten since March 2020 with the past 7 days average have 20 daily cases while daily doses of vaccination being administer by China is around 15 million to 20 million a day.  "
# context= "Notable shift in performance in 2023. While revenue declined due to the completion of major infrastructure projects in 2022, profits grew, driven mainly by the LRT 3 rail construction project. The Division maintained a strong position with a robust client order book, reflecting its leading position in the industry"

# # Format the prompt with actual data using LangChain
# formatted_prompt = prompt_template.format(context=context)

# # Define the system prompt for setting the role
# system_prompt = "You are a financial analyst, your objective is to determine good and bad prospect of a company from it's financial statement and justify why. "

In [67]:
# client = Groq(
#     api_key=os.getenv("GROQ_LLM_API"),
# )

# chat_completion = client.chat.completions.create(
#     messages=[
#         {"role": "system", "content": system_prompt},
#         {"role": "user", "content": formatted_prompt},
#     ],
#     model="llama-3.1-70b-versatile",
# )

# print(chat_completion.choices[0].message.content)

In [35]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("PINECONE_API_KEY")

# configure client
pc = Pinecone(api_key=api_key)

In [52]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

In [53]:
import time

index_name = 'avocado-rag'
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of ada 002
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4}},
 'total_vector_count': 4}

In [38]:
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint="https://minihackathon03.openai.azure.com/",
    api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_version="2023-05-15"
)

### RAG

In [39]:
import fitz  # PyMuPDF

In [46]:
def extract_pages_with_keywords(pdf_path, keywords):
    # Open the PDF file
    document = fitz.open(pdf_path)
    
    # Dictionary to store the page content that contains the keywords
    pages_with_content = {}
    
    # Iterate through each page in the document
    for page_number in range(len(document)):
        page = document[page_number]
        text = page.get_text()  # Extract text from the page
        
        # Check if any of the keywords are in the page text
        if any(keyword.lower() in text.lower() for keyword in keywords):
            pages_with_content[page_number + 1] = text  # Store 1-based page numbers and content
            
    document.close()  # Close the PDF document
    return pages_with_content

# Example usage
paths=[]

pdf_path = "Datasets/maxis-2023.pdf"
keywords = ["Business Segment", "Operation Segment"]

In [92]:
def store_content_in_pinecone(company_name, extracted_pages):
    for page_number, content in extracted_pages.items():
        embedding = embeddings.embed_query(content)
        # Upsert to Pinecone
        index.upsert(vectors=[
            {
                "id": f"{company_name}_{page_number}",
                "values": embedding,
                "metadata": {
                    "company_name": company_name,
                    "content": content,
                    "page_number": page_number,
                }
            }
        ])

extracted_pages = extract_pages_with_keywords(pdf_path, keywords)

# Store content in Pinecone
store_content_in_pinecone("Maxis", extracted_pages)
# print(f"Extracted pages: {extracted_pages}") 

In [95]:
# def query_pinecone(company_name, query):
#     query_embedding = embeddings.embed_query(query)

#     # Query Pinecone for the top K matches using keyword arguments
#     results = index.query(
#         vector=query_embedding, 
#         top_k=5,                  
#         filter={"company_name": company_name} 
#     )

#     return results

# # Example query
# query_result = query_pinecone("Maxis", "What is their business segment")
# for match in query_result['matches']:
#     print(f"ID: {match['id']}")
#     # print(f"Score: {match['score']}")

def query_pinecone(company_name, query):
    query_embedding = embeddings.embed_query(query)

    results = index.query(
        vector=query_embedding, 
        top_k=5,
        filter={"company_name": company_name},
        include_metadata=True
    )

    # Print raw query result
    # print(f"Raw Query Result: {results}")
    
    return results

# Example query
query_result = query_pinecone("Maxis", "What is their business segment")
# for match in query_result['matches']:
#     print(match)

In [102]:
extracted_contexts = [match['metadata']['content'] for match in query_result['matches']]
# match['metadata']['content']
# for x in extracted_contexts:
#     print(x)

In [109]:
print(len(extracted_contexts))

4


In [107]:
# Define the system prompt for setting the role
system_prompt = "You are a financial analyst, your objective is to determine good and bad prospect of a company from it's financial statement and justify why."

def prompt_template(extracted):
    # Define the LangChain prompt template with placeholders
    template = """
    By using the context below, please summarise the business segment of this comapany in a short paragraph.
    
    Context:
    {context}
    
    Please generate the response.
    """
    
    # Create the PromptTemplate
    prompt_template = PromptTemplate(
        input_variables=["context"],
        template=template,
    )
    
    # Format the prompt with actual data using LangChain
    formatted_prompt = prompt_template.format(context=extracted)
    return formatted_prompt

In [104]:
client = AzureOpenAI(
    api_key="2ee9560e08aa4523af24fbd802fe688e",  
    api_version="2024-08-01-preview",
    azure_endpoint ="https://minihackathon03.openai.azure.com/openai/deployments/gpt-4/chat/completions?api-version=2024-08-01-preview"
)

In [105]:
def prompt(system_prompt, formatted):
    response = client.chat.completions.create(
            model="GPT-4o",
            messages=[
                {"role": "system", "content": system_prompt },
                {"role": "user", "content": formatted}
            ]
        )
    return response

In [111]:
templates= []
responses = []
for extracted in extracted_contexts:
    template = prompt_template(extracted)
    templates.append(template)

for template in templates:
    response = prompt(system_prompt,template)
    responses.append(response)
    print("hey: ", response.choices[0].message.content)

hey:  This company is primarily involved in providing digital solutions and accelerating digitalisation for Malaysia's Small and Medium Enterprises (SMEs). They partner with state governments and other entities to create digital ecosystems beneficial for the local communities and micro-SMEs. Their focus also includes next-generation connectivity (with their Hybrid SD-WAN and 5G initiatives) that renders secure and reliable operations for their clients. They have collaborated with various entities, such as petrol stations and plantation networks, providing their advanced IP transmission and network solutions to improve operational efficiency.

In addition, this company is committed to driving e-mobility solutions, notably through the provision of IoT Connectivity Platform for Electric Vehicles. Furthermore, they establish innovation centers that foster digital innovation and transformation for businesses. All these initiatives are geared towards their goal of bridging the digital divide

In [113]:
def prompt_template2(responses):
    # Define the LangChain prompt template with placeholders
    template = """
    By using the context below, please combine and summarise the business segment of this comapany in a concise and clear short paragraph.
    
    Context:
    {responses}
    
    Please generate the response.
    """
    
    # Create the PromptTemplate
    prompt_template = PromptTemplate(
        input_variables=["responses"],
        template=template,
    )
    
    # Format the prompt with actual data using LangChain
    combined_prompt = prompt_template.format(responses=responses)
    return combined_prompt

combined_prompt = prompt_template2(responses)

In [115]:
combined_response= prompt(system_prompt,combined_prompt)

In [116]:
print(combined_response.choices[0].message.content)

Maxis is a leading telecommunications company in Malaysia that is heavily invested in driving digital solutions and digitalisation for Small and Medium Enterprises (SMEs). The company provides advanced IP transmission, network solutions, and initiates 5G services to enhance operational efficiency of businesses. It also promotes IoT services, developing an IoT Connectivity Platform for Electric Vehicles. Additionally, Maxis has diversified into non-traditional segments like broadband and enterprise business. The company secures partnerships aimed at strengthening its hold on the market, thus collaborating with entities like petrol stations, plantation networks, the Malaysian Agricultural Research and Development Institute (MARDI), and property developers to facilitate higher speed broadband and fiber connectivity for various projects. Despite the challenging economic climate and a regulatory landscape that may pose potential risks, Maxis continues to strategize on enhancing their brand,